In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving EmotionalArc&FCS.csv to EmotionalArc&FCS (1).csv
User uploaded file "EmotionalArc&FCS (1).csv" with length 5504110 bytes


In [ ]:
script_df=pd.read_csv('EmotionalArc&FCS.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
script_df['text script'][0]

'[Scene: The Subway, Phoebe is singing for\nchange.]\nPhoebe: (singing) Love is sweet as\nsummer showers, love is a wondrous work of art, but your love oh your love, your love...is\nlike a giant pigeon...crapping on my heart.\xa0 La-la-la-la-la- (some guy gives her some\nchange and to that guy) Thank you. (sings) La-la-la-la...ohhh!\n[Scene: Ross\'s Apartment, the guys are there assembling\nfurniture.]\nRoss: (squatting and reading the instructions) I\'m\nsupposed to attach a brackety thing to the side things, using a bunch of these little worm\nguys. I have no brackety thing, I see no whim guys whatsoever and- I cannot feel my legs.\n(Joey and Chandler are finishing assembling\nthe bookcase.)\nJoey: I\'m thinking we\'ve got a\nbookcase here.\nChandler: It\'s a beautiful thing.\nJoey: (picking up a leftover part) What\'s this?\nChandler: I would have to\nsay that is an \'L\'-shaped bracket.\nJoey: Which goes where?\nChandler: I have no\nidea.\n(Joey checks that Ross is not looking and 

In [ ]:
def split_into_four_parts(script_text):
    # Calculate the total length and the size of each part
    total_length = len(script_text)
    part_size = total_length // 4

    # Split the script into four parts
    parts = [
        script_text[i:i + part_size] for i in range(0, total_length, part_size)
    ]

    # Since the division might not be perfect, we add the remainder to the last part
    if total_length % 4 != 0:
        parts[-1] += script_text[4 * part_size:]

    return parts[:4]

Splitting script into paragraphs

In [ ]:
def split_into_paragraphs(script_text):
    # Splitting by double newlines
    paragraphs = script_text.split('\n\n')
    # Filter out any empty paragraphs
    paragraphs = [para.strip() for para in paragraphs if para.strip() != '']
    return paragraphs

Function to split script into sentences

In [ ]:
import re

# Define a basic function to split the script into sentences using regular expressions
def simple_split_into_sentences(script_text):
    # Regular expression to match sentence endings
    sentence_endings = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
    sentences = re.split(sentence_endings, script_text)
    # Filter out any empty sentences and strip whitespace
    sentences = [sentence.strip() for sentence in sentences if sentence.strip() != '']
    return sentences

To Clean the the paragraphs

In [ ]:
import string

# Define a function to clean text for sentiment analysis
def clean_text_for_sentiment(text):
  # Remove stage directions and character names in brackets
  text = re.sub(r'\[.*?\]', '', text)
  text = re.sub(r'\(.*?\)', '', text)
  # Remove non-letter characters except for basic punctuation
  text = re.sub(r'[^a-zA-Z\s\.\,\!\?]', '', text)
  # Remove multiple spaces and newlines
  text = re.sub(r'\s+', ' ', text)
  # Lowercase all text
  text = text.lower().strip()
  return text

In [ ]:
from textblob import TextBlob

# Function to analyze sentiment of paragraphs using TextBlob
def analyze_sentiment(paragraphs):
    sentiment_scores = []
    for paragraph in paragraphs:
        # Use TextBlob for sentiment analysis
        analysis = TextBlob(paragraph)
        # Polarity score ranges from -1 (most negative) to 1 (most positive)
        sentiment_scores.append(analysis.sentiment.polarity)
    return sentiment_scores

In [ ]:
# Now we'll create a pipeline that uses the previous functions to process the scripts
def process_script_for_emotional_arc(script_text):
    # Split script into paragraphs
    paragraphs = split_into_four_parts(script_text)
    # Clean paragraphs
    cleaned_paragraphs = [clean_text_for_sentiment(para) for para in paragraphs]
    # Analyze sentiment of cleaned paragraphs
    sentiment_scores = analyze_sentiment(cleaned_paragraphs)
    return sentiment_scores


In [ ]:
# Apply the pipeline to each script in the DataFrame and create the 'emotional_arc' column
script_df['emotional_arc'] = script_df['text script'].apply(process_script_for_emotional_arc)
# Display the DataFrame with the new column
script_df[['text script', 'emotional_arc']].head()


,text script,emotional_arc
0,"[Scene: The Subway, Phoebe is singing for\ncha...","[0.12518315018315015, 0.24316202090592334, 0.0..."
1,"[Scene Central Perk, everyone's there.]\nMonic...","[0.09912033066514198, 0.06307988473167046, 0.1..."
2,"[Scene: Chandler and Joey's, Chandler is helpi...","[0.31382320447118234, 0.1014724873426382, -0.0..."
3,"[Scene: Central Perk, Ross and Monica are watc...","[0.1044484812062937, 0.2427170052170053, 0.025..."
4,"[Scene: Central Perk, all six are there.]\nMon...","[0.11739366319444446, 0.12876642036124797, 0.1..."


In [ ]:
script_df['emotional_arc'][10:30]

10    [0.12745834658844302, 0.08001510305614784, 0.0...
11    [0.12241485727813851, 0.18788946607002918, 0.1...
12    [0.0945175316220238, 0.0665162037037037, 0.070...
13    [0.04219671604437231, 0.22490360137765802, 0.1...
14    [0.273410579004329, 0.1739083868253533, 0.1947...
15    [0.11078281349797746, 0.09384251416860111, 0.1...
16    [0.08595191814764183, 0.18158281368102797, 0.1...
17    [0.28442492319508444, 0.18271527777777777, 0.0...
18    [0.20669854375696764, 0.10706223835688122, 0.0...
19    [0.09217861422004243, 0.12177501044277358, 0.0...
20    [0.07019037097162098, 0.02523904458114985, -0....
21    [0.14994036575669223, 0.08860164835164826, 0.2...
22    [0.20840824425008092, 0.18090681524547803, 0.1...
23    [0.07327288535722486, 0.19080588248556998, 0.1...
24    [0.21706299885010824, 0.20751396237507347, 0.0...
25    [0.170518278018278, 0.38024453024453014, 0.086...
26    [0.1447959799861974, 0.08730834317197951, 0.03...
27    [0.10947120009620008, 0.20186287477954146,

In [ ]:
files

In [ ]:
def get_speaking_characters(script_lines):
    characters = set()
    for line in script_lines:
        if ':' in line:
            # Extract the character name
            character = line.split(':')[0].strip()
            # Add the character to the set if it's not empty, doesn't contain spaces, and isn't a scene direction
            if character and ' ' not in character and not character.startswith('['):
                characters.add(character)
    return list(characters)


def create_character_sets(script_text):
    lines = script_text.split('\n')
    return [get_speaking_characters(lines[i:i + 40]) for i in range(0, len(lines), 10)]


def find_frequent_character_sets(script_text, min_support=0.5):
    # Create character sets for every 10 lines
    character_sets = create_character_sets(script_text)

    # Apply the TransactionEncoder to get a boolean array
    te = TransactionEncoder()
    te_ary = te.fit(character_sets).transform(character_sets)
    df = pd.DataFrame(te_ary, columns=te.columns_)

    # Apply Apriori algorithm
    frequent_character_sets = apriori(df, min_support=min_support, use_colnames=True)
    return frequent_character_sets
# Apply the functions to the first entry as an example
frequent_characters_example = find_frequent_character_sets(script_df['text script'][0])

# Display the frequent character sets from the example
frequent_characters_example

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.942857,(Chandler)
1,0.800000,(Joey)
2,0.942857,(Monica)
3,0.885714,(Rachel)
4,0.800000,(Ross)
5,0.742857,"(Joey, Chandler)"
6,0.885714,"(Monica, Chandler)"
7,0.828571,"(Rachel, Chandler)"
8,0.742857,"(Ross, Chandler)"
9,0.771429,"(Monica, Joey)"


In [ ]:
def add_top_itemsets_to_df(script_df, itemset_lengths):
    for length in itemset_lengths:  # For each length (3, 4 in your case)
        column_name = f'top_{length}_itemsets'
        script_df[column_name] = None  # Initialize the column

        for index, script in script_df.iterrows():
            # Find the frequent character sets for each script
            frequent_characters = find_frequent_character_sets(script['text script'])
            # Filter by the length of the itemsets
            frequent_characters['length'] = frequent_characters['itemsets'].apply(lambda x: len(x))
            length_sets = frequent_characters[frequent_characters['length'] == length]
            # Get the top 3 sets based on support, ignoring the support value
            top_sets = length_sets.nlargest(3, 'support')['itemsets'].tolist()
            # Add the result to the DataFrame
            script_df.at[index, column_name] = top_sets

    return script_df

# Apply the function to add the top 3 and 4 itemsets to script_df
itemset_lengths = [3, 4]
script_df = add_top_itemsets_to_df(script_df, itemset_lengths)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
script_df['frequent_character_sets'] = script_df['text script'].apply(find_frequent_character_sets)

# Convert the frozensets in 'itemsets' to a regular set, so it can be easily read and accessed
script_df['frequent_character_sets'] = script_df['frequent_character_sets'].apply(lambda df: [set(item) for item in df['itemsets']])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


349
363
361
435
375
337
378
327
351
369
429
406
366
306
385
690
523
394
393
376
325
383
428
401
335
320
328
334
403
359
329
355
626
400
336
1225
634
623
656
535
547
577
608
639
519
551
562
356
300
353
385
402
373
398
376
465
387
451
378
415
361
406
473
456
399
386
428
489
438
413
419
449
356
363
447
380
380
296
382
420
406
393
361
426
445
462
416
414
410
408
402
405
466
416
702
425
384
399
366
388
389
440
426
369
415
412
409
401
425
400
426
399
422
438
401
481
508
893
407
407
432
423
413
424
379
279
480
386
430
414
404
424
851
408
438
447
461
443
430
426
888
436
376
411
426
426
396
402
392
415
393
444
382
509
451
517
567
428
385
420
413
428
439
829
415
428
416
410
412
463
444
400
419
420
428
373
382
302
400
285
348
372
373
433
446
424
374
804
429
425
400
351
412
585
658
493
481
503
418
407
343
2331
22
408
428
452
3741
3249
2269
340
684
604
559
537
820
3663
628
537
593
454
477
663
688
557
540
349
602
968
1225


In [ ]:
script_df[['text script', 'frequent_character_sets']].head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text script,frequent_character_sets
0,"[Scene: The Subway, Phoebe is singing for\ncha...","[{Chandler}, {Joey}, {Monica}, {Rachel}, {Ross..."
1,"[Scene Central Perk, everyone's there.]\nMonic...","[{Monica}, {Rachel}, {Ross}, {Monica, Ross}, {..."
2,"[Scene: Chandler and Joey's, Chandler is helpi...","[{All}, {Chandler}, {Joey}, {Monica}, {Phoebe}..."
3,"[Scene: Central Perk, Ross and Monica are watc...","[{Chandler}, {Joey}, {Monica}, {Phoebe}, {Rach..."
4,"[Scene: Central Perk, all six are there.]\nMon...","[{Chandler}, {Joey}, {Monica}, {Phoebe}, {Rach..."


In [ ]:
script_df=script_df.drop(columns=['frequent_character_sets'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
script_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0.1,Unnamed: 0,text script,Title,Episode,Season,emotional_arc,top_3_itemsets,top_4_itemsets
0,0,0,"[Scene: The Subway, Phoebe is singing for\ncha...",The Pilot-The Uncut Version,1.0,1.0,"[0.12518315018315015, 0.24316202090592334, 0.0...","[(Monica, Rachel, Chandler), (Monica, Joey, Ch...","[(Joey, Monica, Rachel, Chandler), (Monica, Ro..."
1,1,1,"[Scene Central Perk, everyone's there.]\nMonic...",The One With The Sonogram At the End,2.0,1.0,"[0.09912033066514198, 0.06307988473167046, 0.1...","[(Monica, Phoebe, Chandler), (Monica, Joey, Ch...","[(Monica, Joey, Phoebe, Chandler)]"
2,2,2,"[Scene: Chandler and Joey's, Chandler is helpi...",The One With The Thumb,3.0,1.0,"[0.31382320447118234, 0.1014724873426382, -0.0...","[(Monica, Ross, Rachel), (Monica, Ross, Phoebe...","[(Monica, Ross, Rachel, Phoebe), (Monica, Ross..."
3,3,3,"[Scene: Central Perk, Ross and Monica are watc...",The One With George Stephanopoulos,4.0,1.0,"[0.1044484812062937, 0.2427170052170053, 0.025...","[(Ross, Rachel, Chandler), (Monica, Rachel, Ph...","[(Monica, Ross, Rachel, Chandler), (Joey, Ross..."
4,4,4,"[Scene: Central Perk, all six are there.]\nMon...",The One With The East German Laundry Detergant,5.0,1.0,"[0.11739366319444446, 0.12876642036124797, 0.1...","[(Monica, Ross, Joey), (Monica, Joey, Chandler...","[(Rachel, Monica, Ross, Joey)]"


In [ ]:
script_df.to_csv('EmotionalArc&FCS.csv')
files.download('EmotionalArc&FCS.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>